In [1]:
import sys
sys.path.append("/workspace/")

In [2]:
from src.ast_parser import ASTParser, LanguageRepr

In [3]:
lang_repr = LanguageRepr(
    library_path="/workspace/tmp/ast_test/my-languages.so", lang="python"
)

parser = ASTParser(lang_repr)

In [4]:
program = """\
class ASTParser:
    def __init__(self, language_repr: LanguageRepr):
        self.__language_repr = language_repr
        self.parser = language_repr.parser

    def parse_program(
        self, program: str
    ) -> typing.List[typing.Tuple[str, str]]:
        tree = self.parser.parse(bytes(program, "utf8"))
        root_node = TreeNode(tree.root_node)

        program_lines = program.split("\n")
        src_lines: typing.List[str] = []
        ast_lines: typing.List[str] = []

        for node in self.traverse_tree(root_node):
            if (
                node.type == "class_definition"
                or node.type == "function_definition"
            ):
                src, ast = self.__parse_def(node, program_lines)
                src_lines.append(src)
                ast_lines.append(ast)
            elif node.type == "decorated_definition":
                src, ast = self.__parse_decorated_def(node, program_lines)
                src_lines.append(src)
                ast_lines.append(ast)

            if "statement" in node.type or "definition" in node.type:
                src_lines.append(self.find_substring(program_lines, node))
                ast_lines.append(self.parse_node(node, program_lines))
                print(f"NodeType: {node.type}")

        return list(zip(src_lines, ast_lines))

    def parse_root_children(
        self, program: str
    ) -> typing.List[typing.Tuple[str, str]]:
        tree = self.parser.parse(bytes(program, "utf8"))
        program_lines = program.split("\n")

        raw_bodies: typing.List[str] = []
        parsed_children: typing.List[str] = []

        children = deque([TreeNode(node) for node in tree.root_node.children])

        while children:
            root_child = children.popleft()
            if root_child.type == "class_definition":
                block_node = root_child.children[-1].node
                for node in block_node.children[::-1]:
                    children.appendleft(TreeNode(node))

                source_code, parsed_ast = self.__parse_def(
                    root_child, program_lines
                )

                raw_bodies.append(source_code)
                parsed_children.append(parsed_ast)
            elif root_child.type == "function_definition":
                source_code, parsed_ast = self.__parse_def(
                    root_child, program_lines
                )

                raw_bodies.append(source_code)
                parsed_children.append(parsed_ast)
            elif root_child.type == "decorated_definition":
                for node in root_child.children[::-1]:
                    children.appendleft(TreeNode(node))

            raw_bodies.append(self.find_substring(program_lines, root_child))
            parsed_children.append(self.parse_node(root_child, program_lines))

        return list(zip(raw_bodies, parsed_children))\
"""

In [6]:
len(parser.parse_program(program))

50

In [7]:
len(parser.parse_root_children(program))


8